In [17]:
%matplotlib inline
import torch
import torchvision
from torchvision import transforms,models
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from collections import OrderedDict

from XJ_DLClass import RunBuilder
from XJ_DLClass import RunManager
#查准确率
def get_correct(preds,lab):
    return preds.argmax(1).eq(lab).sum().item()

# 数据集预处理 #

In [18]:
datasets_trans = transforms.Compose([
                transforms.ToTensor()
                ,transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])
train_datasets = torchvision.datasets.ImageFolder('../datasets/data_classify/train'
                                            ,datasets_trans
                                            )
test_datasets = torchvision.datasets.ImageFolder('../datasets/data_classify/test'
                                            ,datasets_trans
                                            )
train_dataloader = torch.utils.data.DataLoader(test_datasets
                                          ,batch_size=10

        )
test_dataloader = torch.utils.data.DataLoader(test_datasets
                                          ,batch_size=10
                                          ,drop_last=True

        )


# 微调ResNet50 #

In [19]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=5):
        super(ResNet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        in_features = self.resnet50.fc.in_features
        for param in self.resnet50.parameters():
            param.requires_grad = False
        self.resnet50.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        x = self.resnet50(x)
        return x
network = ResNet50(num_classes=5).cuda()


D:\python3\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\python3\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
params = OrderedDict(
    lr=[.01]
    ,batch_size=[40]
    ,shuffle=[True]
    ,num_workers=[2]
)
m = RunManager()
for run in RunBuilder().get_runs(params):

    optimizer = optim.Adam(network.parameters(),lr=run.lr)

    m.begin_run(run,network,train_dataloader)
    for epoch in range(10):
        m.begin_epoch()
        for batch in train_dataloader:
            img,lab = batch

            preds = network(img.cuda())

            loss = F.cross_entropy(preds, lab.cuda())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            m.track_loss(loss)
            m.track_num_correct(preds,lab.cuda())

        m.end_epoch()
    m.end_run()


,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle,num_workers
0,1,1,16.193106,0.000,0.588532,0.589507,0.01,40,True,2
1,1,2,21.446534,0.200,0.607002,1.203508,0.01,40,True,2
2,1,3,25.043803,0.300,0.602352,1.810861,0.01,40,True,2
3,1,4,29.357632,0.200,0.688821,2.505681,0.01,40,True,2
4,1,5,31.431605,0.475,0.654049,3.164940,0.01,40,True,2
5,1,6,32.417106,0.675,0.721890,3.893404,0.01,40,True,2
6,1,7,34.831707,0.325,0.729268,4.629672,0.01,40,True,2
7,1,8,35.448148,0.825,0.615436,5.251131,0.01,40,True,2
8,1,9,36.261265,0.700,0.598593,5.855648,0.01,40,True,2
9,1,10,37.042073,0.750,0.598784,6.460489,0.01,40,True,2


# 测试 #

In [24]:
correct = 0
for batch in test_dataloader:
        img,lab = batch
        img = img.cuda()
        lab = lab.cuda()
        preds = network(img)
        correct += get_correct(preds,lab)
print('在训练集上的精度为：',correct/len(test_datasets))

在训练集上的精度为： 1.0
